In [4]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool

# 1. Create the model:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# 2. Create the tools:
tools = [get_word_length]

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 3. Create the Prompt:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events and isn't good at calculating word length.",
        ),
        ("user", "{input}"),
        # This is where the agent will write/read its messages from
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [6]:
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function

# 4. Formats the python function tools into JSON schema and binds them to the model:
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

# 5. Setting up the agent chain:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [7]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "How many letters in the word Software?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'Software'}`


8There are 8 letters in the word "Software".

> Finished chain.


{'input': 'How many letters in the word Software?',
 'output': 'There are 8 letters in the word "Software".'}

---

## Callbacks

In [11]:
from langchain.agents import AgentExecutor
from langchain.callbacks import StdOutCallbackHandler

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    callbacks=[StdOutCallbackHandler()], 
    tags=['a-tag'])

agent_executor.invoke({"input": "How many letters in the word Software?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'Software'}`


8There are 8 letters in the word "Software".

> Finished chain.


{'input': 'How many letters in the word Software?',
 'output': 'There are 8 letters in the word "Software".'}